In [154]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [155]:
out='../output/'
inp='../input/monthly/'
pref='idm'

In [156]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,(2020/01/01) 00-01,30,75
2,(2020/01/01) 01-02,30,70
3,(2020/01/01) 02-03,0,0
4,(2020/01/01) 03-04,30,65
...,...,...,...
23,(2022/11/01) 22-23,0,0
24,(2022/11/01) 23-24,0,0
25,Max,0,0
26,Average,0,0


In [157]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
1,(2020/01/01) 00-01,30.0,75.0
2,(2020/01/01) 01-02,30.0,70.0
3,(2020/01/01) 02-03,0.0,0.0
4,(2020/01/01) 03-04,30.0,65.0
5,(2020/01/01) 04-05,0.0,0.0
...,...,...,...
20,(2022/11/01) 19-20,0.0,0.0
21,(2022/11/01) 20-21,0.0,0.0
22,(2022/11/01) 21-22,0.0,0.0
23,(2022/11/01) 22-23,0.0,0.0


In [158]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
1,(2020/01/01) 00-01,30.0,75.0,2020-01-01,0
2,(2020/01/01) 01-02,30.0,70.0,2020-01-01,1
3,(2020/01/01) 02-03,0.0,0.0,2020-01-01,2
4,(2020/01/01) 03-04,30.0,65.0,2020-01-01,3
5,(2020/01/01) 04-05,0.0,0.0,2020-01-01,4
...,...,...,...,...,...
20,(2022/11/01) 19-20,0.0,0.0,2022-11-01,19
21,(2022/11/01) 20-21,0.0,0.0,2022-11-01,20
22,(2022/11/01) 21-22,0.0,0.0,2022-11-01,21
23,(2022/11/01) 22-23,0.0,0.0,2022-11-01,22


In [159]:
dat=dat.drop(['datehour'],1)
dat

C:\Users\kabom\AppData\Local\Temp\ipykernel_1904\4059115739.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dat=dat.drop(['datehour'],1)


,price,volume,date,hour
1,30.0,75.0,2020-01-01,0
2,30.0,70.0,2020-01-01,1
3,0.0,0.0,2020-01-01,2
4,30.0,65.0,2020-01-01,3
5,0.0,0.0,2020-01-01,4
...,...,...,...,...
20,0.0,0.0,2022-11-01,19
21,0.0,0.0,2022-11-01,20
22,0.0,0.0,2022-11-01,21
23,0.0,0.0,2022-11-01,22


In [160]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2020-01-01,30.0,75.0,0,2020,1,1,2
2020-01-01,30.0,70.0,1,2020,1,1,2
2020-01-01,0.0,0.0,2,2020,1,1,2
2020-01-01,30.0,65.0,3,2020,1,1,2
2020-01-01,0.0,0.0,4,2020,1,1,2
...,...,...,...,...,...,...,...
2022-11-01,0.0,0.0,19,2022,11,1,1
2022-11-01,0.0,0.0,20,2022,11,1,1
2022-11-01,0.0,0.0,21,2022,11,1,1


In [161]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.reset_index().groupby(['year','month','day']).sum().drop(['hour','day_of_week'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily=daily.reset_index()
daily['date']=pd.to_datetime(daily.reset_index()[['year', 'month', 'day']])
daily=daily.set_index(['year','month','day'])
daily


volume     value  price_weight_av  price_max  price_min  \
year month day                                                            
2020 1     1     826.0  38450.32        46.550024      100.0        0.0   
           2    1336.0  69778.31        52.229274      105.0        0.0   
           3    1245.0  71393.85        57.344458       85.0        0.0   
           4    1149.0  37270.00        32.436902       65.0        0.0   
           5     756.0  22680.00        30.000000       30.0        0.0   
...                ...       ...              ...        ...        ...   
2022 10    10      0.0      0.00              NaN        0.0        0.0   
           11      0.0      0.00              NaN        0.0        0.0   
           12      0.0      0.00              NaN        0.0        0.0   
           13    203.0   4872.00        24.000000       24.0        0.0   
     11    1       0.0      0.00              NaN        0.0        0.0   

                price_mean  power_max  power_min  power_mean       date  
year month day                                                           
2020 1     1     30.092917       85.0        0.0   34.416667 2020-01-01  
           2     37.863333      173.0        0.0   63.619048 2020-01-02  
           3     28.745417      246.0        0.0   51.875000 2020-01-03  
           4     24.166667      130.0        0.0   47.875000 2020-01-04  
           5     18.750000      142.0        0.0   31.500000 2020-01-05  
...                    ...        ...        ...         ...        ...  
2022 10    10     0.000000        0.0        0.0    0.000000 2022-10-10  
           11     0.000000        0.0        0.0    0.000000 2022-10-11  
           12     0.000000        0.0        0.0    0.000000 2022-10-12  
           13     5.000000       59.0        0.0    8.458333 2022-10-13  
     11    1      0.000000        0.0        0.0    0.000000 2022-11-01  

[986 rows x 10 columns]

In [162]:
hourly['cat']='off'

if (pref=='dam' or pref=='idm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='evening'
    
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2020-01-01,30.0,75.0,0,2020,1,1,2,2250.0,off
2020-01-01,30.0,70.0,1,2020,1,1,2,2100.0,off
2020-01-01,0.0,0.0,2,2020,1,1,2,0.0,off
2020-01-01,30.0,65.0,3,2020,1,1,2,1950.0,off
2020-01-01,0.0,0.0,4,2020,1,1,2,0.0,off
...,...,...,...,...,...,...,...,...,...
2022-11-01,0.0,0.0,19,2022,11,1,1,0.0,evening
2022-11-01,0.0,0.0,20,2022,11,1,1,0.0,standard
2022-11-01,0.0,0.0,21,2022,11,1,1,0.0,standard


In [163]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume     value  price_weight_av  price_max  price_min  \
year month day                                                            
2020 1     1     826.0  38450.32        46.550024      100.0        0.0   
           2    1336.0  69778.31        52.229274      105.0        0.0   
           3    1245.0  71393.85        57.344458       85.0        0.0   
           4    1149.0  37270.00        32.436902       65.0        0.0   
           5     756.0  22680.00        30.000000       30.0        0.0   
...                ...       ...              ...        ...        ...   
2022 10    10      0.0      0.00              NaN        0.0        0.0   
           11      0.0      0.00              NaN        0.0        0.0   
           12      0.0      0.00              NaN        0.0        0.0   
           13    203.0   4872.00        24.000000       24.0        0.0   
     11    1       0.0      0.00              NaN        0.0        0.0   

                price_mean  power_max  power_min  power_mean       date  ...  \
year month day                                                           ...   
2020 1     1     30.092917       85.0        0.0   34.416667 2020-01-01  ...   
           2     37.863333      173.0        0.0   63.619048 2020-01-02  ...   
           3     28.745417      246.0        0.0   51.875000 2020-01-03  ...   
           4     24.166667      130.0        0.0   47.875000 2020-01-04  ...   
           5     18.750000      142.0        0.0   31.500000 2020-01-05  ...   
...                    ...        ...        ...         ...        ...  ...   
2022 10    10     0.000000        0.0        0.0    0.000000 2022-10-10  ...   
           11     0.000000        0.0        0.0    0.000000 2022-10-11  ...   
           12     0.000000        0.0        0.0    0.000000 2022-10-12  ...   
           13     5.000000       59.0        0.0    8.458333 2022-10-13  ...   
     11    1      0.000000        0.0        0.0    0.000000 2022-11-01  ...   

                morning_hour_count  evening_hour_count  off_volume  \
year month day                                                       
2020 1     1                   3.0                 2.0       339.0   
           2                   3.0                 2.0       290.0   
           3                   3.0                 2.0        33.0   
           4                   NaN                 NaN      1129.0   
           5                   NaN                 NaN       756.0   
...                            ...                 ...         ...   
2022 10    10                  3.0                 2.0         0.0   
           11                  3.0                 2.0         0.0   
           12                  3.0                 2.0         0.0   
           13                  3.0                 2.0       203.0   
     11    1                   3.0                 2.0         0.0   

                standard_volume  morning_volume  evening_volume  off_value  \
year month day                                                               
2020 1     1              457.0            30.0             0.0    10170.0   
           2             1046.0             0.0             0.0     8700.0   
           3             1062.0           150.0             0.0      990.0   
           4               20.0             NaN             NaN    35970.0   
           5                NaN             NaN             NaN    22680.0   
...                         ...             ...             ...        ...   
2022 10    10               0.0             0.0             0.0        0.0   
           11               0.0             0.0             0.0        0.0   
           12               0.0             0.0             0.0        0.0   
           13               0.0             0.0             0.0     4872.0   
     11    1                0.0             0.0             0.0        0.0   

                standard_value  morning_value  evening_value  
year month

In [164]:
daily.to_csv(out + pref + '_daily.csv')

In [165]:
hourly.to_csv(out + pref + '_hourly.csv')

In [166]:
hourly_flat=hourly.reset_index().set_index(['date'])
hourly_slim=hourly_flat[['hour','price','volume','cat']]
hourly_slim

,hour,price,volume,cat
date,,,,
2020-01-01,0,30.0,75.0,off
2020-01-01,1,30.0,70.0,off
2020-01-01,2,0.0,0.0,off
2020-01-01,3,30.0,65.0,off
2020-01-01,4,0.0,0.0,off
...,...,...,...,...
2022-11-01,19,0.0,0.0,evening
2022-11-01,20,0.0,0.0,standard
2022-11-01,21,0.0,0.0,standard


In [167]:
hourly_slim.to_csv(out + pref + '_hourly_slim.csv')

In [168]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_stdev','standard_price_stdev','off_price_stdev','morning_price_stdev','evening_price_stdev']]=daily.groupby(['year','month']).std().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

C:\Users\kabom\AppData\Local\Temp\ipykernel_1904\3018208690.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)


,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2020,1,985.483871,53468.051935,62.000000,0.0,30.383836,129.129032,0.000000,42.932335,...,46.0,54.255634,32.142199,64.906243,80.040513,85.233918,36.100946,34.159420,57.170819,14.869565
1,2020,2,297.620690,15747.605862,46.041667,0.0,14.563890,47.931034,0.689655,12.838013,...,40.0,52.911664,30.720552,69.224777,85.000000,85.623188,12.481132,0.750000,15.400000,17.250000
2,2020,3,504.096774,29568.772258,61.077500,0.0,23.454969,60.354839,1.290323,22.529647,...,44.0,58.656936,29.743800,63.369498,82.994055,97.889149,13.198198,35.015152,29.951852,18.954545
3,2020,4,655.900000,23886.040000,43.063750,0.0,17.750851,76.333333,3.500000,28.067937,...,44.0,36.417198,23.218280,35.603486,53.411492,72.083612,20.437700,29.136364,35.870370,38.000000
4,2020,5,457.935484,15577.333548,35.041667,0.0,12.046970,54.516129,4.032258,19.906874,...,42.0,34.016437,24.323109,39.389279,50.286008,82.774892,17.754386,7.555556,27.883459,5.500000
5,2020,6,241.033333,12926.578333,58.800000,0.0,13.635815,48.700000,0.000000,10.413214,...,66.0,53.629837,20.293351,50.900064,59.951392,111.301964,2.482201,10.560606,20.460317,9.257576
6,2020,7,598.354839,29771.077742,49.967000,0.0,28.122721,91.064516,0.000000,26.503840,...,69.0,49.754888,19.661564,49.353281,65.314062,124.972321,11.479233,64.289855,37.965649,8.304348
7,2020,8,736.903226,36085.918387,53.639000,0.0,24.522003,85.129032,1.548387,32.504263,...,63.0,48.969684,20.850128,47.707332,61.719536,117.634118,13.246291,54.015873,54.128000,22.936508
8,2020,9,1096.666667,47867.866667,51.950000,0.0,28.154159,123.133333,0.533333,47.708413,...,44.0,43.648511,22.930684,47.449751,59.971471,90.000000,27.479233,73.939394,69.770370,13.204545
9,2020,10,1262.129032,59679.031935,66.250000,0.0,32.263411,135.580645,3.258065,53.932220,...,44.0,47.284414,25.152415,56.729911,65.490669,97.754583,46.343558,65.954545,64.888087,38.431818


In [169]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2020,1,985.483871,53468.051935,62.000000,0.0,30.383836,129.129032,0.000000,42.932335,...,46.0,54.255634,32.142199,64.906243,80.040513,85.233918,36.100946,34.159420,57.170819,14.869565
1,2020,2,297.620690,15747.605862,46.041667,0.0,14.563890,47.931034,0.689655,12.838013,...,40.0,52.911664,30.720552,69.224777,85.000000,85.623188,12.481132,0.750000,15.400000,17.250000
2,2020,3,504.096774,29568.772258,61.077500,0.0,23.454969,60.354839,1.290323,22.529647,...,44.0,58.656936,29.743800,63.369498,82.994055,97.889149,13.198198,35.015152,29.951852,18.954545
3,2020,4,655.900000,23886.040000,43.063750,0.0,17.750851,76.333333,3.500000,28.067937,...,44.0,36.417198,23.218280,35.603486,53.411492,72.083612,20.437700,29.136364,35.870370,38.000000
4,2020,5,457.935484,15577.333548,35.041667,0.0,12.046970,54.516129,4.032258,19.906874,...,42.0,34.016437,24.323109,39.389279,50.286008,82.774892,17.754386,7.555556,27.883459,5.500000
5,2020,6,241.033333,12926.578333,58.800000,0.0,13.635815,48.700000,0.000000,10.413214,...,66.0,53.629837,20.293351,50.900064,59.951392,111.301964,2.482201,10.560606,20.460317,9.257576
6,2020,7,598.354839,29771.077742,49.967000,0.0,28.122721,91.064516,0.000000,26.503840,...,69.0,49.754888,19.661564,49.353281,65.314062,124.972321,11.479233,64.289855,37.965649,8.304348
7,2020,8,736.903226,36085.918387,53.639000,0.0,24.522003,85.129032,1.548387,32.504263,...,63.0,48.969684,20.850128,47.707332,61.719536,117.634118,13.246291,54.015873,54.128000,22.936508
8,2020,9,1096.666667,47867.866667,51.950000,0.0,28.154159,123.133333,0.533333,47.708413,...,44.0,43.648511,22.930684,47.449751,59.971471,90.000000,27.479233,73.939394,69.770370,13.204545
9,2020,10,1262.129032,59679.031935,66.250000,0.0,32.263411,135.580645,3.258065,53.932220,...,44.0,47.284414,25.152415,56.729911,65.490669,97.754583,46.343558,65.954545,64.888087,38.431818


In [170]:
monthly=monthly.set_index(['year','month'])

In [171]:
monthly['off_price_diff']=monthly['off_price_mean']-monthly['off_price_mean'].shift(1)
monthly['morning_price_diff']=monthly['morning_price_mean']-monthly['morning_price_mean'].shift(1)
monthly['standard_price_diff']=monthly['standard_price_mean']-monthly['standard_price_mean'].shift(1)
monthly['evening_price_diff']=monthly['evening_price_mean']-monthly['evening_price_mean'].shift(1)

#monthly['off_price_log_diff']=np.log10(monthly['off_price_mean'])-np.log10(monthly['off_price_mean']).shift(1)
#monthly['morning_price_log_diff']=np.log10(monthly['morning_price_mean'])-np.log10(monthly['morning_price_mean']).shift(1)
#monthly['standard_price_log_diff']=np.log10(monthly['standard_price_mean'])-np.log10(monthly['standard_price_mean']).shift(1)
#monthly['evening_price_log_diff']=np.log10(monthly['evening_price_mean'])-np.log10(monthly['evening_price_mean']).shift(1)

In [172]:
monthly.to_csv(out + pref + '_monthly.csv')

In [173]:
monthly_filt=monthly #.loc[monthly['year']<2020]
daily_filt=daily.reset_index() #.loc[daily.reset_index()['year']<2020]

In [174]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean'}, inplace=True)

calmonthly[['price_median', 'off_price_median','standard_price_median', 'morning_price_median', 'evening_price_median']]=daily_filt.groupby(['month']).median().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]
calmonthly[['price_stdev', 'off_price_stdev','standard_price_stdev', 'morning_price_stdev', 'evening_price_stdev']]=daily_filt.groupby(['month']).std().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]
calmonthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]
calmonthly[['off_power','standard_power', 'morning_power', 'evening_power']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_power','standard_power', 'morning_power', 'evening_power']]

calmonthly

C:\Users\kabom\AppData\Local\Temp\ipykernel_1904\2677402245.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'],1)


,month,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,off_price_mean,...,evening_volume,value,off_hour_count,standard_hour_count,morning_hour_count,evening_hour_count,off_power,standard_power,morning_power,evening_power
0,1,42228.460968,53.293779,69.512688,0.376344,27.719124,102.075269,0.107527,33.467089,12.734722,...,705.333333,1.309082e+06,333.666667,271.000000,65.0,43.333333,25.805579,41.954183,48.386473,16.321601
1,2,20638.394118,54.285043,56.226471,0.282353,16.673795,61.341176,0.235294,15.803725,9.302514,...,514.666667,5.847545e+05,309.333333,250.333333,60.0,40.000000,11.518847,21.097043,17.966667,12.866667
2,3,23434.855161,51.524138,56.767957,0.763441,19.189252,62.010753,1.301075,19.417524,11.223415,...,796.666667,7.264805e+05,322.333333,277.333333,68.0,45.333333,12.623374,22.320843,35.420509,17.593544
3,4,21459.929888,56.404765,65.734382,0.872697,18.740338,58.932584,1.516854,18.610005,11.033325,...,1073.666667,6.366446e+05,314.666667,265.000000,64.0,42.666667,11.578486,25.078359,22.698990,25.332576
4,5,14753.620000,54.854665,65.662069,0.528736,13.304935,49.471264,1.436782,13.316578,5.619410,...,445.333333,4.278550e+05,321.333333,250.333333,60.0,40.000000,8.430717,19.311795,14.716138,11.808201
5,6,34692.437011,84.576441,124.356322,1.737816,31.744831,61.390805,0.666667,15.336563,14.974200,...,1365.666667,1.006081e+06,305.000000,242.000000,63.0,63.000000,8.166116,21.798263,16.635832,22.039607
6,7,33785.428182,80.454063,123.097792,0.000000,31.382459,75.177922,0.000000,21.099150,12.471951,...,948.200000,8.671593e+05,265.000000,216.000000,56.0,56.000000,10.083707,23.999152,31.722881,17.854985
7,8,87886.456897,73.636033,130.152414,1.701149,41.961470,105.482759,2.149425,41.698933,15.836495,...,3081.666667,2.548707e+06,309.000000,238.666667,62.0,62.000000,27.523436,57.898450,58.853776,51.452520
8,9,33623.059889,58.360131,80.575111,0.744444,22.409729,82.944444,0.177778,26.864775,10.080845,...,487.333333,1.008692e+06,313.000000,270.000000,66.0,44.000000,14.806177,40.614815,36.449495,11.075758
9,10,33797.426267,65.783458,68.226667,1.617467,22.072198,78.293333,1.346667,27.240095,10.593543,...,605.666667,8.449357e+05,273.000000,218.666667,52.0,34.666667,18.322877,26.547909,31.524531,13.810606


In [175]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [176]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

C:\Users\kabom\AppData\Local\Temp\ipykernel_1904\1619064361.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)


price     volume       value
month hour                                  
1     0      9.950145  12.811594  419.640000
      1     12.309275  13.927536  474.090145
      2     12.800000  17.275362  587.507246
      3     13.098750  19.444444  660.129583
      4     13.805054  23.053763  762.579355
...               ...        ...         ...
12    19    19.870968   6.564516  536.048387
      20    16.822581   9.048387  595.500000
      21    20.080645  14.451613  894.032258
      22     6.806452   4.532258  182.338710
      23    10.066129  12.161290  449.335484

[288 rows x 3 columns]

In [177]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')

In [178]:
yearly_calmonthly_hours=hourly.groupby(['year','month','hour']).mean()
yearly_calmonthly_hours=yearly_calmonthly_hours.drop(['day','day_of_week'],1)
yearly_calmonthly_hours

C:\Users\kabom\AppData\Local\Temp\ipykernel_1904\4270116260.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  yearly_calmonthly_hours=yearly_calmonthly_hours.drop(['day','day_of_week'],1)


price     volume        value
year month hour                                   
2020 1     0     18.608696  28.260870   892.739130
           1     20.565217  27.043478   901.130435
           2     20.347826  31.260870  1032.739130
           3     23.458333  37.583333  1241.375000
           4     22.838710  45.451613  1441.516129
...                    ...        ...          ...
2022 11    19     0.000000   0.000000     0.000000
           20     0.000000   0.000000     0.000000
           21     0.000000   0.000000     0.000000
           22     0.000000   0.000000     0.000000
           23     0.000000   0.000000     0.000000

[840 rows x 3 columns]

In [179]:
yearly_calmonthly_hours.to_csv(out + pref + '_yearly_calmonthly_hours.csv')

In [180]:
from statsmodels.tsa.stattools import pacf

In [181]:
pacfs=pd.DataFrame(data={'standard': pacf(monthly.reset_index()['standard_price_mean'], nlags=14)})
pacfs['evening']= pacf(monthly.reset_index()['evening_price_mean'], nlags=14)
pacfs['morning']= pacf(monthly.reset_index()['morning_price_mean'], nlags=14)
pacfs['off']= pacf(monthly.reset_index()['off_price_mean'], nlags=14)
pacfs

,standard,evening,morning,off
0,1.000000,1.000000,1.000000,1.000000
1,0.092442,0.518648,0.158344,0.444684
2,-0.019162,-0.355947,-0.112908,0.047360
3,-0.161865,-0.208644,-0.075374,-0.002058
4,0.008181,-0.139160,0.129230,0.171862
5,0.029503,-0.037365,0.052156,0.006231
6,-0.099631,-0.175491,-0.008637,-0.143300
7,0.273600,-0.033298,0.015452,-0.211293
8,-0.236767,-0.163770,-0.227415,0.047350
9,0.233583,-0.043715,0.242551,0.026177


In [182]:
pacfs.to_csv(out + pref + '_pacfs.csv')